In [1]:
import pandas as pd
import os
import glob
import traceback
from datetime import datetime
import pandas as pd
from plotly.graph_objs import Scatter,Layout
import plotly
import plotly.offline as py
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots
plotly.offline.init_notebook_mode(connected=True)
import plotly.express as px
import gezi
from gezi import tqdm 
import pymp
import time
import qgrid
from sklearn.metrics import roc_auc_score
from multiprocessing import Manager, cpu_count
from projects.feed.rank.utils.doc_info.KV import KV

In [2]:
%%html
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:10000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>

In [3]:
mark = 'tuwen'
info_dir = f'/search/odin/publicData/CloudS/rank/infos/{mark}/common'
info_dir = f'/home/gezi/tmp/rank/exps/{mark}/v25/dlrm'
hour = 2020031523
info_path = f'{info_dir}/infos/{hour}/valid.csv'
print(info_path)
df = pd.read_csv(info_path)
# df2 = pd.read_csv()
# df = df.rename({'duration':'dur'})
df['dur'] = df.duration
df.head()

/home/gezi/tmp/rank/exps/tuwen/v25/dlrm/infos/2020031523/valid.csv


,mid,duration,ori_lr_score,abtest,rea,page_time,product_data,distribution,impression_time,position,read_completion_rate,docid,pred,pred_click,pred_dur,dur
0,22e4860143045636971,0,0.169433,4,901,9,sgsapp,3116,1584285667,5,0.0,open_doc_prod22783156,0.015589,0.063328,0.440680,0
1,c546861514044876314,0,0.153747,5,993,28,sgsapp,5253,1584284283,7,0.0,20314j0l4k7aCR,0.019589,0.156120,0.113574,0
2,31ea866350031574579,0,0.129212,4,912,14,sgsapp,1439,1584284377,3,0.0,open_doc_prod22743688,0.011989,0.057496,0.362143,0
3,9ceb861313044693617,0,0.064033,4,908,311,sgsapp,3116,1584285751,1,0.0,2030300l3Kavdz,0.006489,0.108853,0.045471,0
4,4eb0865044035995721,0,0.417808,5,931,78,sgsapp,3057,1584285883,1,0.0,20200312A0LEW000,0.018489,0.170358,0.089335,0


In [4]:
kv = KV()
titles = kv.get_titles(list(set(df.docid.values)))
df['title'] = df.docid.apply(lambda x: titles.get(x, ''))

get_titles: 100%|##########| 347/347 [00:16<00:00, 22.03it/s]


In [5]:
pd.set_option('display.max_rows', 1000)

In [6]:
if 'article_page_time' not in df.columns:
  df.article_page_time = df.impression_time
df['page_time'] = (df.impression_time - df.article_page_time) / 60 / 60 / 24

In [7]:
mean_duration = df[df.dur > 0].dur.mean()

In [8]:
mean_duration

86.32951348128273

In [9]:
from projects.feed.rank.src import util
df['cold'] = df.rea.apply(lambda x: int(x) in util.cold_starts)
# df = df[df.product_data=='sgsapp']

In [10]:
cold_mids = set(df[df.cold==True].mid.values)

In [11]:
len(set(df.mid.values))

16544

In [12]:
len(cold_mids)

1226

In [13]:
cols = ['position', 'docid', 'title', 'dur', 'ori_lr_score', 'pred', 'pred_click', 'pred_dur', 'cold']
def debug(mid=None, cold=False):
  global cols
  if mid is None:
    if not cold:
      mid = df.mid.sample(1).values[0]
    else:
      mid = df[df.mid.isin(cold_mids)].mid.sample(1).values[0]
  print('mid:', mid)
  if 'hour' in df.columns:
    cols += ['hour']
  d = df[df.mid==mid]
  start_time = d.impression_time.min()
  d['time'] = d.impression_time.apply(lambda x: int((x - start_time) / 60))
  cols += ['time']
  d = d.sort_values(['impression_time', 'position'])[cols].reset_index(drop=True)
#   return d.sort_values(['ori_lr_score'], ascending=False)[cols].reset_index(drop=True)
  return d

In [14]:
d = debug()
d

mid: c15a865924042539994


,position,docid,title,dur,ori_lr_score,pred,pred_click,pred_dur,cold,time
0,0,2031380l3wa7Dh,一人毁了900万人努力，“毒王”引爆郑州，钟南山担心的事发生了,0,1.333333,0.012689,0.114847,0.108338,True,0
1,1,2020890x1Xs9Pz,马云保护多年的老婆终于曝光，网友：怪不得马云有钱都没学坏!,0,0.201969,0.008889,0.125495,0.054737,True,0
2,2,open_doc_prod20484366,清朝一幅女鬼图，很多网友都不敢看，放大10倍你看她在干什么!,0,1.666667,0.006889,0.101416,0.056383,True,0
3,4,open_doc_prod22000842,过安检时，女孩子有没有想过自己的“隐私”，会在仪器上暴露,0,2.000000,0.020189,0.243099,0.052442,False,0
4,6,2030470G3LCBaV,三个字证明你是哪个省的，不许出现地名!(*︾▽︾),0,0.787923,0.000789,0.018804,0.056221,False,0
5,8,open_doc_prod21387984,全网都骂她“滚出娱乐圈”，却又在偷偷收藏她的私照,0,0.165556,0.003789,0.063619,0.057905,True,0
6,10,20227n0l2alYU6,巴基斯坦突然拒绝与中国合作 背后的原因中国人真的该反思,0,0.160676,0.003589,0.047489,0.092165,True,0
7,12,20216j0x1jWaYw,拐点已经到来!教育部发重磅通知回应，家长乐了，却有一个坏消息,0,0.153549,0.001589,0.028311,0.069777,True,0
8,13,open_doc_prod20743892,警报再次拉响，第二新冠爆发国已出现，感染人数比其余国家都多,0,0.137461,0.000289,0.006315,0.080753,True,0
9,14,open_doc_prod22616552,安阳杀夫凶案：蛇蝎女子为谋财，与母亲合谋，3年毒杀4任新婚丈夫,0,0.106580,0.001089,0.014693,0.140562,True,0


In [15]:
d = d.sort_values(['pred'], ascending=False)[cols].reset_index(drop=True)
d

,position,docid,title,dur,ori_lr_score,pred,pred_click,pred_dur,cold,time
0,4,open_doc_prod22000842,过安检时，女孩子有没有想过自己的“隐私”，会在仪器上暴露,0,2.000000,0.020189,0.243099,0.052442,False,0
1,6,20313e0l2ya34O,出国避难的明星又回来了?,0,0.180635,0.016189,0.102205,0.190779,False,4
2,0,20200312A0LEW000,这地方开学第一天，578名学生发烧，万人请假，中央发布紧急通知,0,0.217440,0.013589,0.122493,0.106154,True,4
3,0,2031380l3wa7Dh,一人毁了900万人努力，“毒王”引爆郑州，钟南山担心的事发生了,0,1.333333,0.012689,0.114847,0.108338,True,0
4,1,2020890x1Xs9Pz,马云保护多年的老婆终于曝光，网友：怪不得马云有钱都没学坏!,0,0.201969,0.008889,0.125495,0.054737,True,0
5,2,open_doc_prod20484366,清朝一幅女鬼图，很多网友都不敢看，放大10倍你看她在干什么!,0,1.666667,0.006889,0.101416,0.056383,True,0
6,1,open_doc_prod22345064,乌克兰美女为何“愁嫁”这些生理缺陷，当地男人直呼接受不了,0,0.116135,0.006189,0.070841,0.095170,True,5
7,2,open_doc_prod20541806,范冰冰晒浓妆照，无意间暴露沙发上的人影，网友评论炸了!,0,0.173780,0.004889,0.084770,0.048427,True,4
8,0,open_doc_prod22735972,韩国疯了!意大利崩了!日本慌了!中国亮了：该出手时就出手!,0,0.140710,0.004489,0.031256,0.276410,True,5
9,0,open_doc_prod19760450,女子因感冒被隔离，出来后得知自己信用卡账单哭红了眼,0,0.145779,0.004489,0.046036,0.134433,True,5


In [16]:
d = d.sort_values('ori_lr_score', ascending=False)
d['idx'] = range(len(d))
d[d.dur!=0 ].idx.mean()

nan

In [17]:
d = d.sort_values('pred', ascending=False)
d['idx'] = range(len(d))
d[d.dur!=0 ].idx.mean()

nan

In [18]:
roc_auc_score(d.dur !=0, d.ori_lr_score)

ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.

In [ ]:
roc_auc_score(d.dur !=0, d.pred)

In [ ]:
d = d.sort_values('ori_lr_score', ascending=False)
roc_auc_score((d.dur != 0).values[:20], d.ori_lr_score.values[:20])

In [ ]:
d = d.sort_values('pred', ascending=False)
roc_auc_score((d.dur != 0).values[:20], d.pred.values[:20])

In [ ]:
d = d.sort_values('pred_click', ascending=False)
roc_auc_score((d.dur != 0).values[:20], d.pred_click.values[:20])

In [ ]:
qgrid.set_grid_option('forceFitColumns', False)

In [ ]:
qgrid.show_grid(d)

In [ ]:
len(set(df.mid.values))

In [ ]:
d = df[['mid', 'impression_time']].groupby('mid', as_index=False).agg({'impression_time':'nunique'})

In [ ]:
d = df[['mid', 'impression_time']].groupby('mid', as_index=False).agg({'impression_time':'count'})

In [ ]:
df['mid'].value_counts()

In [ ]:
len(d[d.impression_time<10])

In [ ]:
px.histogram(d[d.impression_time<100], 'impression_time')

In [ ]:
df[['mid', 'impression_time']]

In [ ]:
df['impression'] = df.mid + '\t' + df.impression_time.astype(str)

In [ ]:
impressions = set(df[['mid', 'impression_time', 'impression']].groupby('mid', as_index=False).first().impression.values)

In [ ]:
len(df[df.impression.isin(impressions)].sort_values('mid'))

In [ ]:
len(set(df[df.cold==True].docid.values))

In [ ]:
len(df[df.cold==True])

In [ ]:
util.cold_starts

In [ ]:
df[df.cold==True].head()

In [ ]:
d = df[['mid', 'dur']]
d.dur = d.dur != 0

In [ ]:
d2 = d.groupby(['mid']).max()

In [ ]:
d2[d2.duration==False].reset_index().mid